In [2]:
# !nvcc --version

In [3]:
# ### For Colab, install dependencies.

# !pip install mxnet-cu101==1.7 
# !pip install gluonts
# !pip install fredapi
# !pip install stats-can
# !pip install --upgrade scikit-learn

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# %cd /content/drive/MyDrive/Colab Notebooks/foodprice-forecasting
# !pwd

In [6]:
import pandas as pd
pd.set_option('precision', 3)
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

import numpy as np
import pickle
import data

import importlib
importlib.reload(data)

from data import update_expl_data, update_target_data, food_categories, preprocess_expl
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

import os

import mxnet as mx
from gluonts.model.n_beats import NBEATSEnsembleEstimator
from gluonts.mx import Trainer
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import make_evaluation_predictions


In [7]:
"""
Set sample rate. In this notebook, all data will be resampled at the chosen frequency.
'MS' : Monthly (Month Start)
'W' : Weekly
'D' : Daily
"""

year_period = {'MS': 12, 'W': 52, 'D': 365}
frequency = 'MS'
one_year = year_period[frequency]
output_path = "./output/nbeats_202110"
if not os.path.exists(output_path):
    os.mkdir(output_path)

## Load Data Using APIs

In [8]:
"""
Load food CPI data from January 1986 to the most recently available data.
"""

# foodprice_df = update_target_data(food_categories, './data_files/food_cpi.csv')
foodprice_df = pd.read_csv("./data_files/food_cpi.csv")
foodprice_df = foodprice_df.set_index("REF_DATE")
foodprice_df.index = pd.DatetimeIndex(foodprice_df.index)
foodprice_df = foodprice_df.resample(frequency).mean().interpolate()
foodprice_df

,Bakery and cereal products (excluding baby food),Dairy products and eggs,"Fish, seafood and other marine products",Food purchased from restaurants,Food,"Fruit, fruit preparations and nuts",Meat,Other food products and non-alcoholic beverages,Vegetables and vegetable preparations
REF_DATE,,,,,,,,,
1986-01-01,69.3,70.9,60.6,59.1,67.3,76.0,65.1,77.5,76.0
1986-02-01,70.3,70.8,61.3,59.1,66.9,77.6,64.2,78.1,68.4
1986-03-01,70.6,71.1,61.3,59.3,67.0,79.2,64.2,78.6,66.2
1986-04-01,71.3,71.0,61.4,59.7,67.7,82.2,63.6,79.5,71.1
1986-05-01,71.2,71.4,61.9,59.9,68.2,83.5,64.0,79.8,75.3
...,...,...,...,...,...,...,...,...,...
2021-05-01,157.8,146.6,147.6,163.5,156.6,143.9,175.4,141.6,153.8
2021-06-01,157.7,145.3,146.2,163.9,156.8,144.5,176.7,142.2,153.4
2021-07-01,157.9,146.4,146.6,165.2,157.6,141.7,180.9,141.9,154.8


# NBEATS Model and Experiments

## Data Splitting

For each such candidate forecast, we should record any uncertainty/confidence metrics it provides, and evaluation metrics for that same model configuration over the test set. i.e. When model configuration XYZ was used to forecast Meat prices over the test set (with that data not being used for training or validation!) - what were its evaluation metrics on the withheld data? We should report this consistently for ALL EXPERIMENTS. 

For all models, we will use the following "simulated" report dates. This is a form of cross validation over time. We train a model up to each cutoff date, and then produce and evaluate 18-month forecasts. We can then collect each model's validation metric, take the mean, and use this to do model selection for the final forecast (or ensemble of forecasts!).

In [9]:
report_sim_dates = ["2015-07-01", "2016-07-01", "2017-07-01", "2018-07-01", "2019-07-01", "2020-07-01"]

In [10]:
sim_train_dates = {}
sim_valid_dates = {}

for date in report_sim_dates:
    sim_train_dates[date] = foodprice_df.index[foodprice_df.index <= date]
    sim_valid_dates[date] = foodprice_df.index[(foodprice_df.index > date) & (foodprice_df.index <= (pd.to_datetime(date) + pd.DateOffset(months=18)))]

## Fitting and Evaluating a Single NBEATS Model: Example Using All Food Prices

In [11]:
dataset_df = foodprice_df.T
dataset_df

REF_DATE,1986-01-01,1986-02-01,1986-03-01,1986-04-01,1986-05-01,1986-06-01,1986-07-01,1986-08-01,1986-09-01,1986-10-01,...,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01
Bakery and cereal products (excluding baby food),69.3,70.3,70.6,71.3,71.2,71.1,71.7,71.9,71.7,71.1,...,156.4,154.2,157.1,156.8,156.2,157.8,157.7,157.9,158.5,158.1
Dairy products and eggs,70.9,70.8,71.1,71.0,71.4,71.1,71.3,71.5,71.8,71.8,...,141.5,141.6,143.1,144.9,146.1,146.6,145.3,146.4,148.3,148.0
"Fish, seafood and other marine products",60.6,61.3,61.3,61.4,61.9,62.0,62.2,62.7,63.1,63.6,...,144.7,143.4,143.9,144.9,145.1,147.6,146.2,146.6,146.8,147.1
Food purchased from restaurants,59.1,59.1,59.3,59.7,59.9,60.0,60.6,60.9,60.9,61.3,...,161.6,162.6,162.9,162.6,163.2,163.5,163.9,165.2,165.9,165.9
Food,67.3,66.9,67.0,67.7,68.2,68.4,69.2,69.5,69.9,70.2,...,153.6,155.0,155.6,155.5,155.4,156.6,156.8,157.6,158.0,158.5
"Fruit, fruit preparations and nuts",76.0,77.6,79.2,82.2,83.5,83.1,84.8,86.7,83.8,82.9,...,140.0,140.9,143.4,142.4,141.9,143.9,144.5,141.7,142.5,141.5
Meat,65.1,64.2,64.2,63.6,64.0,64.9,66.5,67.8,71.3,71.5,...,170.0,171.9,169.5,170.2,173.5,175.4,176.7,180.9,182.1,184.8
Other food products and non-alcoholic beverages,77.5,78.1,78.6,79.5,79.8,79.9,80.2,80.2,80.8,81.0,...,136.5,139.0,139.5,141.5,140.5,141.6,142.2,141.9,141.7,144.3
Vegetables and vegetable preparations,76.0,68.4,66.2,71.1,75.3,74.1,75.7,71.9,66.6,70.7,...,157.1,162.5,163.8,157.4,151.1,153.8,153.4,154.8,152.2,150.0


In [12]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

metrics = {
    'r2_score': r2_score,
    'mae': mean_absolute_error,
    'mape': mean_absolute_percentage_error,
    'mse': mean_squared_error,
    'rmse': rmse
}
def get_prophet_df(foodprice_df, food_category, dates):
    df = foodprice_df[food_category][dates]
    df = df.reset_index()
    df = df.rename({'REF_DATE':'ds', food_category:'y'}, axis=1)
    return df

In [13]:

def train_eval_nbeats(report_sim_date, prediction_length=18):

    report_train_dates = sim_train_dates[report_sim_date]
    report_valid_dates = sim_valid_dates[report_sim_date]
    
    # train dataset: cut the last window of length "prediction_length", add "target" and "start" fields
    train_ds = ListDataset(
        [{'target': x, 'start': report_sim_date} for x in dataset_df[list(report_train_dates)].values],
        freq='MS'
    )

    valid_ds_report = ListDataset(
        [{'target': x, 'start': report_sim_date} for x in dataset_df[list(report_train_dates) + list(report_valid_dates)].values],
        freq='MS'
    )

    estimator = NBEATSEnsembleEstimator(
        prediction_length=prediction_length,
        #context_length=7*prediction_length,
        meta_bagging_size = 3,  # 3
        meta_context_length = [prediction_length * m for m in [3,5,7] ], 
        meta_loss_function = ['sMAPE'], 
        num_stacks = 30,
        widths= [512],
        freq="MS",
        trainer=Trainer(
                    # learning_rate=6e-4,
                    #clip_gradient=1.0,
                    epochs=50,
                    # num_batches_per_epoch=1000,
                    # batch_size=16,
                    ctx=mx.context.gpu()
                )

    )

    predictor = estimator.train(train_ds)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=valid_ds_report,  # test dataset
        predictor=predictor,  # predictor
    )

    forecasts = list(forecast_it)
    tss = list(ts_it)
    all_fc_dates = list(report_train_dates) + list(report_valid_dates)

    all_food_metrics = {}
    food_forecasts = {}

    for target_index in range(len(forecasts)):

        # Get food price category
        foodprice_category = foodprice_df.columns[target_index]

        # plot actual
        fig, ax = plt.subplots(figsize=(8,3))
        ax.scatter(all_fc_dates, foodprice_df[foodprice_category][all_fc_dates], color='black')

        # plot forecast
        forecast_entry = forecasts[target_index]
        ax.plot(report_valid_dates, forecast_entry.mean[:len(report_valid_dates)], color='C0')

        plt.title(f"{foodprice_category}, {report_sim_date}")
        plt.grid()
        plt.show()

        fc_metrics = {}
        for metric_name, metric_fn in metrics.items():
            print(metric_name, metric_fn)
            y_true = foodprice_df[foodprice_category][report_valid_dates]
            y_pred = forecast_entry.mean[:len(report_valid_dates)]
            score = metric_fn(y_true=y_true, y_pred=y_pred)
            print(metric_name, score)
            fc_metrics[metric_name] = score

        # fc_metrics = pd.Series({metric_name: metric_fn(y_true=foodprice_df[foodprice_category][report_valid_dates], y_pred=forecast_entry.mean[:len(report_valid_dates)]) for metric_name, metric_fn in metrics.items()})
        # print(fc_metrics)

        all_food_metrics[foodprice_category] = fc_metrics
        food_forecasts[foodprice_category] = pd.Series(forecast_entry.mean[:len(report_valid_dates)], index=report_valid_dates, name=foodprice_category)

        # print(valid_df)
        # print(valid_forecast)

    all_forecasts = pd.DataFrame(food_forecasts)
    all_forecasts.to_csv(f"{output_path}/forecasts_{report_sim_date}.csv")

    return all_food_metrics, all_forecasts

In [14]:
# all_valid_metrics = {}
# all_forecasts = {}

# for report_sim_date in report_sim_dates:
#     valid_metrics, forecasts = train_eval_nbeats(report_sim_date)
#     all_valid_metrics[report_sim_date] = valid_metrics
#     all_forecasts[report_sim_date] = forecasts

In [15]:
# valid_metrics_concat = {}

# all_valid_metrics.keys()

# for report_date, valid_scores in all_valid_metrics.items():
#     valid_metrics_concat[report_date] = pd.DataFrame(valid_scores).T
# index = valid_metrics_concat[report_date].index
# columns = valid_metrics_concat[report_date].columns
# scores = [df.values for date, df in valid_metrics_concat.items()]
# mean_scores = pd.DataFrame(np.array(scores).mean(axis=0), index=index, columns=columns)
# mean_scores.to_csv(f"{output_path}/mean_fc_valid_metrics.csv")
# mean_scores

## Fit Models Using All Data To Produce Final Forecast

In [16]:
cutoff_date = "2021-09-01"
prediction_length = 18

train_dates = foodprice_df.loc[foodprice_df.index <= cutoff_date].index

train_ds = ListDataset(
    [{'target': x, 'start': train_dates[-1]} for x in dataset_df[list(train_dates)].values],
    freq='MS'
)

estimator = NBEATSEnsembleEstimator(
    prediction_length=prediction_length,
    #context_length=7*prediction_length,
    meta_bagging_size = 3,  # 3
    meta_context_length = [prediction_length * m for m in [3,5,7] ], 
    meta_loss_function = ['sMAPE'], 
    num_stacks = 30,
    widths= [512],
    freq="MS",
    trainer=Trainer(
                # learning_rate=6e-4,
                #clip_gradient=1.0,
                epochs=50,
                num_batches_per_epoch=100,
                # batch_size=16,
                ctx=mx.context.cpu()
            )

)

predictor = estimator.train(train_ds)

forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_ds,  # train dataset
    predictor=predictor,  # predictor
)

forecasts = list(forecast_it)
all_fc_dates = pd.date_range(pd.to_datetime(cutoff_date) + pd.DateOffset(months=1), pd.to_datetime(cutoff_date) + pd.DateOffset(months=prediction_length), freq='MS')

all_food_metrics = {}
food_forecasts = {}

for target_index in range(len(forecasts)):

    # Get food price category
    foodprice_category = foodprice_df.columns[target_index]

    # plot actual
    fig, ax = plt.subplots(figsize=(8,3))
    ax.plot(train_dates, foodprice_df[foodprice_category][train_dates], color='black')

    # plot forecast
    forecast_entry = forecasts[target_index]
    ax.plot(all_fc_dates, forecast_entry.mean[:len(all_fc_dates)], color='C0')

    plt.title(f"{foodprice_category}, October 2021 Forecast")
    plt.grid()
    plt.show()

    food_forecasts[foodprice_category] = pd.Series(forecast_entry.mean[:len(all_fc_dates)], index=all_fc_dates, name=foodprice_category)

all_forecasts = pd.DataFrame(food_forecasts)
all_forecasts.to_csv(f"{output_path}/fc_final.csv")

/home/ethan/anaconda3/envs/forecasting/lib/python3.8/site-packages/gluonts/dataset/common.py:323: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(string, freq=freq)
/home/ethan/anaconda3/envs/forecasting/lib/python3.8/site-packages/gluonts/dataset/common.py:326: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  if isinstance(timestamp.freq, Tick):
/home/ethan/anaconda3/envs/forecasting/lib/python3.8/site-packages/gluonts/dataset/common.py:337: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  return timestamp.freq.rollforward(timestamp)


TRAINER:gluonts.mx.trainer._base.Trainer(add_default_callbacks=True, batch_size=None, callbacks=None, clip_gradient=10.0, ctx=mxnet.context.Context("cpu", 0), epochs=50, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=100, patience=10, weight_decay=1e-08)


  0%|          | 0/100 [00:00<?, ?it/s]/home/ethan/anaconda3/envs/forecasting/lib/python3.8/site-packages/gluonts/transform/split.py:36: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  return _shift_timestamp_helper(ts, ts.freq, offset)
[09:35:45] src/base.cc:49: GPU context requested, but no GPUs found.
/home/ethan/anaconda3/envs/forecasting/lib/python3.8/site-packages/gluonts/transform/split.py:36: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  return _shift_timestamp_helper(ts, ts.freq, offset)
  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
all_forecasts

Error: Kernel is dead

## Predicted Change in CPI By Category

For the report, we usually express forecasts as the predicted percentage change, overall for the next year. We can do this by comparing the mean forecasted CPI for 2022 to the mean (known and predicted) values for 2021.